In [1]:
import numpy as np
from torch import nn
from torch.nn import functional as F
import torch
from torch import optim
from torchvision import datasets,transforms
from torchinfo import summary


from trainer import Trainer
from tester import Tester
from utlis import visualize_data,show_misclassified_images,device,is_cuda
from viz import plot_class_distribution, plot_confusion_matrix, plot_curves
from dataloader import CIFAR10Dataset,CIFAR10DataLoader
from models import ConvLayer,TransBlock,DepthwiseConvLayer
from transformations import train_transforms,test_transforms



atrain_dataset = CIFAR10Dataset(root='../data/',train=True,Atransforms=train_transforms,download=False)
atest_dataset  = CIFAR10Dataset(root='../data/',train=False,Atransforms=test_transforms,download=False)
acifar = CIFAR10DataLoader(batch_size=512,is_cuda_available=True)
atrain_loader, atest_loader = acifar.get_loader(atrain_dataset,atest_dataset)

In [2]:
import wandb

In [3]:
import os
os.environ['WANDB_API_KEY'] = '47aafcfd9a3f65de61e49bc0362c1b8553d2d9e1'
wandb.login(force=True)

wandb: Currently logged in as: muthukamalan98. Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(name='session 9', project='CIFAR10 ',notes='This is a introductory project')

In [5]:

class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self,norm='BN',drop=0.01):
      super(Net,self).__init__()

      ##Block 1
      self.conv1 = nn.Conv2d(3, 16, 3, padding=1,bias=False)
      self.norm1 = self.select_norm(norm,16)
      self.conv2 = nn.Conv2d(16, 32, 3, padding=1,bias=False)
      self.norm2 = self.select_norm(norm,32)
      self.conv3 = nn.Conv2d(32, 32, 3, padding=1,bias=False)
      self.norm3 = self.select_norm(norm,32)
      self.dilated_conv1 = nn.Conv2d(32,32,3,stride = 2,dilation=2,padding=0,bias=False)
      self.dilated_norm1 = self.select_norm(norm,32)
        
      ##Block 2
      self.conv4 = nn.Conv2d(32, 32, 3, padding=1,bias=False)
      self.norm4 = self.select_norm(norm,32)
      self.conv5 = nn.Conv2d(32, 52, 3, padding=1,bias=False)
      self.norm5 = self.select_norm(norm,52)
      self.dilated_conv2 = nn.Conv2d(52,64,3,stride = 2,dilation=2,padding=0,bias=False)
      self.dilated_norm2 = self.select_norm(norm,64)
        
      ## Block 3
      #nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, groups=16)
      self.depthwise_conv1 = nn.Conv2d(64, 64,1,stride = 1,groups = 64, padding=0,bias=False)
      self.depthwise_norm1 = self.select_norm(norm,64)
      self.conv6 = nn.Conv2d(64, 64, 3, padding=1,bias=False)
      self.norm6 = self.select_norm(norm,64)
      self.strided_conv1 = nn.Conv2d(64,64,1,stride = 2,padding=1,bias=False)
      self.strided_norm1 = self.select_norm(norm,64)
        
      ## Block 4
      self.conv7 = nn.Conv2d(64, 64, 3, padding=1,bias=False)
      self.norm7 = self.select_norm(norm,64)
      self.conv8 = nn.Conv2d(64,10,3,stride = 1, padding=1,bias=False)

      self.drop = nn.Dropout2d(drop)
      self.gap = nn.AvgPool2d(4)      

    def forward(self, x):
        x = self.drop(self.norm1(F.relu(self.conv1(x))))
        x = self.drop(self.norm2(F.relu(self.conv2(x))))
        x = self.drop(self.norm3(F.relu(self.conv3(x))))
        x = self.drop(self.dilated_norm1(F.relu(self.dilated_conv1(x))))

        x = self.drop(self.norm4(F.relu(self.conv4(x))))
        x = self.drop(self.norm5(F.relu(self.conv5(x))))
        x = self.drop(self.dilated_norm2(F.relu(self.dilated_conv2(x))))

   

        x = self.drop(self.depthwise_norm1(F.relu(self.depthwise_conv1(x))))
        x = self.drop(self.norm6(F.relu(self.conv6(x))))
        x = self.drop(self.strided_norm1(F.relu(self.strided_conv1(x))))
         
        x = self.drop(self.norm7(F.relu(self.conv7(x))))
        x = self.conv8(x)
        x = self.gap(x)
      
        x = x.view(-1, 10)
        return F.log_softmax(x,dim=-1) 
  
        
    def select_norm(self, norm, channels,groupsize=2):
        if norm == 'BN':
            return nn.BatchNorm2d(channels)
        elif norm == 'LN':
            return nn.GroupNorm(1,channels)
        elif norm == 'GN':
            return nn.GroupNorm(groupsize,channels)            

In [6]:
model = Net(norm="BN",drop=0.01)
summary(model=model,input_size=(1,3,32,32))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [1, 10]                   --
├─Conv2d: 1-1                            [1, 16, 32, 32]           432
├─BatchNorm2d: 1-2                       [1, 16, 32, 32]           32
├─Dropout2d: 1-3                         [1, 16, 32, 32]           --
├─Conv2d: 1-4                            [1, 32, 32, 32]           4,608
├─BatchNorm2d: 1-5                       [1, 32, 32, 32]           64
├─Dropout2d: 1-6                         [1, 32, 32, 32]           --
├─Conv2d: 1-7                            [1, 32, 32, 32]           9,216
├─BatchNorm2d: 1-8                       [1, 32, 32, 32]           64
├─Dropout2d: 1-9                         [1, 32, 32, 32]           --
├─Conv2d: 1-10                           [1, 32, 14, 14]           9,216
├─BatchNorm2d: 1-11                      [1, 32, 14, 14]           64
├─Dropout2d: 1-12                        [1, 32, 14, 14]           --
├─Con

In [7]:
# prev_test_loss = float('inf')

optimizer = optim.SGD(params=model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode='min',factor=0.1,patience=3,verbose=True)
criterion = nn.NLLLoss()
trainer = Trainer(model=model, train_loader=atrain_loader, optimizer=optimizer, criterion=criterion, device=device)
tester = Tester(model=model, test_loader=atest_loader,criterion=criterion, device=device)
wandb.watch(model)
for epoch in range(1, 10):
    train_acc,train_loss,_= trainer.train(epoch=epoch)
    test_acc ,test_loss = tester.test()
    # if prev_test_loss>test_loss:
    #     torch.save(obj=model.state_dict(),f='./bmodels/model.pth')
    wandb.log({
        "Epoch": epoch,
        "Train Loss": train_loss,
        "Train Acc": train_acc,
        "Valid Loss": test_loss,
        "Valid Acc": test_acc
    })
    scheduler.step(test_loss)
wandb.finish()

EPOCH = 1 | LR = 0.1 | Loss = 1.52 | Batch = 97 | Accuracy = 33.32: 100%|███| 98/98 [00:50<00:00,  1.95it/s]


Test set: Average loss: 0.0031, Accuracy: 4391/10000 (43.91%)


EPOCH = 2 | LR = 0.1 | Loss = 1.41 | Batch = 97 | Accuracy = 43.22: 100%|███| 98/98 [00:49<00:00,  1.99it/s]


Test set: Average loss: 0.0026, Accuracy: 5295/10000 (52.95%)


EPOCH = 3 | LR = 0.1 | Loss = 1.42 | Batch = 97 | Accuracy = 48.76: 100%|███| 98/98 [00:48<00:00,  2.02it/s]


...

EPOCH = 8 | LR = 0.1 | Loss = 1.02 | Batch = 97 | Accuracy = 61.35: 100%|███| 98/98 [00:30<00:00,  3.19it/s]


Test set: Average loss: 0.0017, Accuracy: 7059/10000 (70.59%)


EPOCH = 9 | LR = 0.1 | Loss = 1.02 | Batch = 97 | Accuracy = 62.55: 100%|███| 98/98 [00:30<00:00,  3.17it/s]


Test set: Average loss: 0.0016, Accuracy: 7281/10000 (72.81%)


Epoch,▁▂▃▄▅▅▆▇█
Train Acc,▁▃▅▆▆▇▇██
Train Loss,▁▁▁▁▁▁▁▁▁
Valid Acc,▁▃▄▅▆▆▇▇█
Valid Loss,█▆▅▄▄▃▂▁▁
Epoch,9
Train Acc,62.55
Train Loss,0.0
Valid Acc,72.81
Valid Loss,0.00155
